Purpose: clean and aggregate multi-tab leader tracker into one sheet
- link https://docs.google.com/spreadsheets/d/186JSFYCzweoyct0RtCZcYwp6nSFZb22zYBHVYHqKTPA/edit#gid=0

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 75)

import matplotlib.pyplot as plt

# TO DO:
 - Put Leader tracker in a form to be imported into EA regularly. (main priority)
     - Clean up names (done)
     - Standardize leadership status (done)
     - Clean up Phone numbers (done)
     - Clean up zip codes (done)
     - Remove duplicates
     - Assign Organization, Membership (done)
     - Identify institutions that may need to be cleaned before initial upload (bolded in csv)
 - Need counts and analyses that are exportable (later, nobody has asked for this lol)
     - How many (overall, by organizer, by region?, by institution?, by leadership status?, by zip?, by recruited by?, title): people, phones, emails, zips, institutions, leadership status, core team, ROL, recruited by
     - Viz: bar chart - #ppl per organizer, 
     
# Questions
 - Removing duplicates by email, first, and last. Or email and last (see who these ppl are)?
 - How to make df.drop_duplicates() not take NaN into account? (Tianyi)
      - try: df[(~df.duplicated()) | (df['col'].isnull())] according to the internet?
      - include organizer in subset?
 - When I upload to EA want to assign organizer to people I think. What if a person is on multiple organizers lists? See if more than one person can be assigned to a person in EA?
 - Column counts by recruited by, in order of how many names they have
     - Daniel has a lot of records with only emails and no names
 - Can switch leadership status mappings to be the ones without numbers
 - Does Edwin have a list in our AN setup? is he in BFO?

In [2]:
# Read all tabs you want from leader tracker and write in df
dataset = r'C:\Users\OOC User\Documents\OOC\Data\Leader Tracking\2022\OOC 2022 Leaders_jan4_2022.xlsx'
data = pd.ExcelFile(dataset) 
df1 = pd.DataFrame() #where the imported data will go after formatting


#read in data
for i in range(3, len(data.sheet_names)): #don't need first 2 or last tabs in file
    df_temp = pd.read_excel (dataset, sheet_name = data.sheet_names[i], header = 3, dtype = {'Phone':str})#Read an organizers sheet
    df_temp = df_temp.loc[:,:'Notes & Reflections'] #Take first column through notes column only
    df_temp['Organizer'] = data.sheet_names[i]#add column for organizer
    df_temp = df_temp.loc[~(df_temp['First Name'].isnull() & df_temp['Last Name'].isnull() & df_temp['E-Mail'].isnull()),:] #Remove rows with null first/last name/Email
    df_temp = df_temp.iloc[:-1] #Remove last row
    df1 = pd.concat([df1, df_temp]) #add df_temp to df1

# #strip everything? this didnt work lol
# for x in df1:
#     df1[x] = df1[x].astype(str).str.strip()
# df1.fillna(value='nan', inplace=True)
    
    
#drop index
df1 = df1.reset_index(drop=True)
df1.loc[df1['Organizer']=='Molly - School Funding', 'Organizer'] = 'Molly'

In [3]:
df1.Organizer.value_counts().sort_index()

Akii        34
Celeste     38
Daniel     118
Derrick     32
Edwin      403
Gail        11
Jacob      207
Kalesha     43
Kalia       38
Molly       20
Rachael    267
Reegie      48
Robin      116
Sammi       19
Tami        44
Name: Organizer, dtype: int64

# Core Team, Relational Organizing Leader Mapping

~~~
0 > No
1 > Yes
~~~


In [4]:
df1.loc[:, 'Relational Organizing Leader'] = df1.loc[:, 'Relational Organizing Leader'].map({0.0:'No', 1.0:'Yes'})
df1.loc[:, 'Core Team'] = df1.loc[:, 'Core Team'].map({0.0:'No', 1.0:'Yes'})

# Leadership Status Mapping

~~~
'Prospect': '1-Prospect'
'Volunteer Leader': '2-Volunteer'
'Member':'3-Member Leader
'Super Leader':'4-Super Leader'
'0-N/A': BLANK
~~~

In [5]:
#Leadership status mapping
df1.loc[df1['Leadership Status']=='Prospect', 'Leadership Status'] = '1-Prospect'
df1.loc[df1['Leadership Status']=='Volunteer Leader', 'Leadership Status'] = '2-Volunteer'
df1.loc[df1['Leadership Status']=='Member', 'Leadership Status'] = '3-Member Leader'
df1.loc[df1['Leadership Status']=='Super Leader', 'Leadership Status'] = '4-Super Leader'
df1.loc[df1['Leadership Status']=='0-N/A', 'Leadership Status'] = np.nan

# Phone Numbers
- Remove non-digit characters
- Output numbers for follow-up if:
    - number length not 7, 10, or 11
    - number length is 11 but doesn't start with U.S. country code 1

In [6]:
# Checking phone numbers
#Remove all non digits characters
df1.loc[:, 'Phone'] = df1.loc[:, 'Phone'].str.replace("\D", '', regex=True) #remove nondigit characters

In [7]:
df1.loc[:, 'Phone'].value_counts()

93769436150    4
93723283130    3
93778998220    3
93735077630    3
30127589430    3
              ..
9374787895     1
93747754500    1
6149465444     1
9375121502     1
6148854628     1
Name: Phone, Length: 552, dtype: int64

In [8]:
# Names with non-letter characters
# Output first/last names that contain characters that aren't letters
first_new = df1['First Name'].str.strip().str.contains("[^a-z]", case=False, regex=True)
last_new = df1['Last Name'].str.strip().str.contains("[^a-z]", case=False, regex=True)
names_new = df1.loc[first_new | last_new, ['First Name', 'Last Name', 'E-Mail', 'Phone', 'Zip Code', 'Organizer']]
names_new = names_new.reset_index(drop=True)

# #read in previous names
# # prev_date = '7.25.2021' #should be defined above
# names_old = pd.read_csv(f'Old_Names_{prev_date}.csv')
# # names_old = pd.read_csv('Old_Names_7.25.2021.csv')
# names_old = names_old.reset_index(drop=True)

# common = pd.merge(left=names_new, right=names_old, on=['First Name','Last Name', 'Organizer'], how='outer', indicator=True)
# new_only = common.loc[common['_merge']=='left_only',:]
# old_only = common.loc[common['_merge']=='right_only',:]

# #Are there old names missing from new names list?
# if len(old_only) > 0:
#     print("WARNING: Old names are missing from new names list\nWARNING: Investigate \"Old_names_to_check.csv\"\n\n")
# #     old_only.to_csv('Old_names_to_check.csv', index=False)
    
#     #print Old Names to check
#     print('Old names to check\n')
#     display(old_only)
# else:
#     print("All old names were found on the new names list, no old names to check\n\n")

    
# #Are there any new names missing from the old names list?
# if len(new_only) > 0:
#     print("WARNING: New names that aren't on old list\nWARNING: Investigate \"New_names_to_check.csv\"\n\n")
# #     new_only.to_csv('New_names_to_check.csv', index=False)
    
#     #print Old Names to check
#     print('New names to check\n\n')
#     display(new_only)
    
# else:
#     print('All new names found in old names list, no new names to check\n\n')

# #output all new names to be checked against for next time
# names_new.to_csv(f'Old_Names_{date}.csv', index=False)
# print(f'Prev. Date for next time: {date}\n\n')

# Organization and Membership Mapping

    Caring Economy (name TBD)  - none
        • Child Care - Tami
        • Unemployment - Kalia
    Amos Project - none
        • Amos Cinci - Jacob, Daniel
        • Amos Columbus - Derrick H.
        • Amos Cleveland - none
    Ohio Student Association  - none
        • OSA Cleveland -  Rachael
        • OSA Columbus - Akii
        • OSA Dayton - Kalesha
        • OSA Cincinnati - none, chech with Rachael if she has a cincy list
    Building Freedom Ohio  - none
        • BFO Cleveland - Robin
        • BFO Cinci - Celeste
        - BFO Dayton - Edwin

In [9]:
#Assigning Organization

# WHAT TO DO WHEN ORGANIZERS CHANGE? IE JANIAH HAS AN ASSIGNED ORG, WHAT HAPPENS WHEN SHE LEAVES OOC? 
# WHAT MEMBER SHOULD JANIAH BE? CINCINNATI?

#Caring Economy
df1.loc[(df1['Organizer'].str.contains("Tami|Janiah")), 'Organization'] = 'Caring Economy'
#AMOS
df1.loc[(df1['Organizer'].str.contains("Daniel|Derrick")), 'Organization'] = 'AMOS'
#OSA
df1.loc[(df1['Organizer'].str.contains("Rachael|Akii|Kalesha")), 'Organization'] = 'OSA'
#BFO
df1.loc[(df1['Organizer'].str.contains("Edwin")), 'Organization'] = 'BFO'   

#Assigning Membership
#Caring Economy
df1.loc[df1['Organizer']=="Tami", 'Membership'] = 'CE: Child Care'
df1.loc[df1['Organizer']=="Janiah", 'Membership'] = 'CE: Unemployment'
#AMOS
df1.loc[df1['Organizer']=="Daniel", 'Membership'] = 'AMOS: Cincinnati'
df1.loc[df1['Organizer']=="Derrick", 'Membership'] = 'AMOS: Columbus'
#OSA
df1.loc[df1['Organizer']=="Rachael", 'Membership'] = 'OSA: Cleveland'
# df1.loc[df1['Organizer']=="Rachael1", 'Membership'] = 'OSA: Cleveland' #Ask about this first
df1.loc[df1['Organizer']=="Akii", 'Membership'] = 'OSA: Columbus'
df1.loc[df1['Organizer']=="Kalesha", 'Membership'] = 'OSA: Dayton'
#BFO
# df1.loc[df1['Organizer']=="Robin", 'Membership'] = 'BFO: Cleveland'
df1.loc[df1['Organizer']=="Edwin", 'Membership'] = 'BFO: Dayton'

In [10]:
df1.Organizer.value_counts()

Edwin      403
Rachael    267
Jacob      207
Daniel     118
Robin      116
Reegie      48
Tami        44
Kalesha     43
Celeste     38
Kalia       38
Akii        34
Derrick     32
Molly       20
Sammi       19
Gail        11
Name: Organizer, dtype: int64

In [11]:
#Organizers without an organization/membership
#Assign organization? Move records to another list? 
no_org = df1.loc[df1['Organization'].isnull(), ['Organizer', 'Organization']]['Organizer'].value_counts()
no_org

Jacob      207
Robin      116
Reegie      48
Celeste     38
Kalia       38
Molly       20
Sammi       19
Gail        11
Name: Organizer, dtype: int64

# Title Mapping
~~~
Acceptable mappings and titles:
    Pastor
    Rev./Reverend >> Rev
    Bishop
    Dr. >> Dr
    Rabbi
    Evangelist?
    Rev Dr
~~~



In [12]:
acceptable_titles = ['Pastor', 'Rev', 'Bishop', 'Dr', 'Rabbi', 'Evangelist', 'Rev Dr']
df1['Title'] = df1['Title'].astype(str).str.strip()
df1.loc[df1['Title']=='nan','Title']=np.nan

#Title Mapping
df1.loc[df1['Title'].isin(["Rev.", "Reverend"]), ['Title']] = 'Rev'
df1.loc[df1['Title'].isin(["Dr."]), ['Title']] = 'Dr'

#New titles to consider?
titles_new = df1.loc[~(df1['Title'].isin(acceptable_titles)) & (df1['Title'].notnull()), :]
titles_new
if len(titles_new) > 0:
    print("The following TITLES have been output for follow-up, investigate 'Titles_to_check.csv': \n")
    display(titles_new)
    titles_new.to_csv(f'Titles_to_check.csv', index=False)
else:
    print("No new titles to consider")

The following TITLES have been output for follow-up, investigate 'Titles_to_check.csv': 



,First Name,Last Name,E-Mail,Phone,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Organizer,Title,Organization,Membership
1419,Susie,Kaeser,NaN,NaN,NaN,League of Women Voters,NaN,No,No,NaN,NaN,Molly,Education Chair,NaN,NaN
1421,Taniesha,Head,NaN,NaN,NaN,OOC,NaN,No,No,NaN,NaN,Molly,"mother of 7, students in Cleveland Metropolita...",NaN,NaN
1422,Heather,Ekechi,parmacouncilpta@gmail.com,NaN,NaN,PTA Parma,NaN,No,No,NaN,NaN,Molly,Parma Council of PTAs President,NaN,NaN
1423,Diana,Dickson,Diana Dickson <ddsdiana@hotmail.com>,NaN,NaN,Retired OAPSE,NaN,No,No,NaN,NaN,Molly,Retired OAPSE,NaN,NaN
1424,Jean,Melvin,NaN,6148854628,NaN,Public Education Partners,NaN,No,No,NaN,NaN,Molly,Retired Hilliard Teachers,NaN,NaN
1425,Maureen,Reedy,NaN,NaN,NaN,Public Education Partners,NaN,No,No,NaN,NaN,Molly,Retired Columbus (?),NaN,NaN
1426,Josephene Shelton Towns,NaN,NaN,NaN,NaN,Ohio Federation of Teachers,NaN,No,No,NaN,NaN,Molly,School social worker Cleveland Heights,NaN,NaN
1427,Shauntina Thornton,NaN,NaN,NaN,NaN,Ohio Federation of Teachers,NaN,No,No,NaN,NaN,Molly,teacher Cleveland Metropolitan Schools,NaN,NaN
1428,"Luke Strohm,",NaN,NaN,NaN,NaN,Ohio Federation of Teachers,NaN,No,No,NaN,NaN,Molly,teacher Grand Valley School District,NaN,NaN
1430,Maxwell,Briggs,max44121@yahoo.com>,NaN,NaN,NaN,NaN,No,No,NaN,NaN,Molly,Cleveland Heights,NaN,NaN


# Zip Codes

In [13]:
# Output zip if nondigit character or first digit is not 4
# Check against zips from previous run

#convert to string
df1['Zip Code'] = df1['Zip Code'].astype(str).str.strip()
#first 5 characters
df1['Zip Code'] = df1['Zip Code'].str.slice(0,5)
# #Remove non digit characters
# non_digits = df1['Zip Code'].str.contains("[^0-9]", case=False, regex=True)
# not_na = ~df1['Zip Code'].str.contains('nan')
# non_digits = df1.loc[non_digits & not_na, 'Zip Code']
# # display(non_digits)
# #first digit is not 4
# not_4 = ~df1['Zip Code'].str.startswith('4', na=False)
# not_4 = df1.loc[not_4 & not_na, 'Zip Code']
# # display(not_4)
# #put it all together
# zips_new = df1.loc[df1['Zip Code'].isin(list(not_4)) | df1['Zip Code'].isin(list(non_digits)), ['First Name', 'Last Name', 'E-Mail', 'Phone', 'Zip Code', 'Organizer']]
# zips_new = zips_new.reset_index(drop=True)


# #read in previous zips
# # prev_date = '7.25.2021' #should be defined above already
# zips_old = pd.read_csv(f'old_zips_{prev_date}.csv')
# # names_old = pd.read_csv('Old_Names_7.25.2021.csv')
# zips_old = zips_old.reset_index(drop=True)

# common_zips = pd.merge(left=zips_new, right=zips_old, on=['First Name','Last Name', 'Zip Code', 'Organizer'], how='outer', indicator=True)
# new_only = common_zips.loc[common_zips['_merge']=='left_only',:]
# old_only = common_zips.loc[common_zips['_merge']=='right_only',:]

# #Are there old zips missing from new zips list?
# if len(old_only) > 0:
#     print("WARNING: Old zips are missing from new zips list, hopefully they were cleaned in the leader tracker? \"old_zips_to_check.csv\"")
# #     old_only.to_csv('old_zips_to_check.csv', index=False)
    
#     #print Old Names to check
#     print('Old zips to check:\n')
#     display(old_only)
# else:
#     print("All old zips were found on the new zips list, no old zips to check\n\n")
    
# #Are there any new zips missing from the old zips list?
# if len(new_only) > 0:
#     print("\n\nWARNING: New zips that aren't on old list, investigate if they're legit \"New_zips_to_check.csv\"")
# #     new_only.to_csv('New_zips_to_check.csv', index=False)
    
#     #print Old zips to check
#     print('New zips to check:\n\n')
#     display(new_only)
    
# else:
#         print('All new zips found in old names list, no new zips to check\n\n')
        
# #output all new zips to be checked against for next time
# zips_new.to_csv(f'Old_zips_{date}.csv', index=False)
# print(f'Prev. Date for next time: {date}\n\n')

In [14]:
#Remove @ohorganizing.org emails  
df = df1.loc[~df1['E-Mail'].str.contains('@ohorganizing.org', na=False), :] 

# Duplicates

In [15]:
#Drop exact duplicates

print(f'Started with {str(len(df))} records\n\n')

exact_dups = df[df.duplicated()]
email_name_dups = df[df.duplicated(subset = ['E-Mail', 'First Name', 'Last Name'])]
email_last_dups = df[df.duplicated(subset = ['E-Mail', 'Last Name'])]

#Remove exact duplicates
print(f'Exact duplicates removed: {len(exact_dups)}')
df = df.drop_duplicates()

#Remove duplicates with matching email/first/last
print(f'Email/First/Last duplicates removed: {len(email_name_dups)}')
df = df.drop_duplicates(subset = ['E-Mail', 'First Name', 'Last Name'])

# # Remove duplicates with matching email/last
# print(f'Email/Last duplicates removed: {len(email_last_dups)}\n\n')
# df = df.drop_duplicates(subset = ['E-Mail', 'Last Name'])

print(f'{len(df)} records remaining')

Started with 1431 records


Exact duplicates removed: 11
Email/First/Last duplicates removed: 45
1386 records remaining


In [16]:
#Duplicate emails
for x, y in zip(df['E-Mail'].value_counts(), df['E-Mail'].value_counts().index):
    if x>1:
        print(f'{y} '+str(x))

allanka@miamioh.edu 3
patdillard13@gmail.com 3
ceh_0769@yahoo.com 3
revdl3rd@aol.com 2
Asiarosegibbs@gmail.com 2
mbadgley831@att.net 2
bartlettbl@udayton.edu 2
newdlcntr@aol.com 2
beelermr@mail.uc.edu 2
alanagf@newvoicespittsburgh.org 2
sue@pretrial.org 2
imcwillporter@gmail.com 2
jennifer@urbanvillagechurch.org 2
diebelalice@gmail.com 2
gilberm2@mail.uc.edu 2
pastorsmcameron@gmail.com 2
gaylisacarr@yahoo.com 2
barbflu47@yahoo.com 2
stgirl8086@gmail.com 2
dionrgreen@gmail.com 2
rcartergreen@sbcglobal.net 2
Tycurington@aol.com 2
geej@mcohio.org 2
jagrace@wowway.com 2
taristi6@gmail.com 2
lydia.boadaclista@yahoo.com 2
keswes5160@yahoo.com 2
feliciafg1983@gmail.com 2
ehess@cppsadmin.org 2
itscmh@sbcglobal.net 2
James@PACEmentoring.org 2
shirtle@sbcglobal.net 2
brielhope@gmail.com 2
shanae.hespeth@caresource.com 2
kerrixs5@gmail.com 2
shakurahmad@hotmail.com 2
weecaredc@gmail.com 2
carloshill198@gmail.com 2
jjames@uuma.org 2
aanderson@udayton.edu 2
laproche@gmail.com 2
mtussey3860@gmail.co

In [17]:
# which organizers have the duplicate emails?
test = df['E-Mail'].value_counts().to_frame().rename({'E-Mail':'count'}, axis=1)
test['E-Mail'] = test.index

test = pd.merge(left=test, right=df, on='E-Mail', how='left')

test[['count', 'E-Mail', 'First Name', 'Last Name', 'Organizer']]

,count,E-Mail,First Name,Last Name,Organizer
0,3,allanka@miamioh.edu,?,?,Edwin
1,3,allanka@miamioh.edu,Kathryn,Allan,Edwin
2,3,allanka@miamioh.edu,Darius,Beckham,Edwin
3,3,patdillard13@gmail.com,Samantha,Bowden,Edwin
4,3,patdillard13@gmail.com,Rebecca,Cochran,Edwin
...,...,...,...,...,...
1079,1,rick4peace@gmail.com,Rick,Polhamus,Edwin
1080,1,larrypostell483@yahoo.com,Larry,Postell,Edwin
1081,1,Sprince@coloursofliving.com,Samuel,Prince,Edwin
1082,1,jpruce1@udayton.edu,Joel,Pruce,Edwin


# Institution - Not going into EA yet

In [18]:
#Exported csv of all institutions. Bolded ones need to be looked at. Can they be combined? Should some be removed?
# pd.DataFrame(df.loc[:,'Institution'].value_counts().index.sort_values()).to_csv('Institutions.csv')

# View the data

In [19]:
# Look at the data, should probably go after duplicates

df.head()

# df.describe(include='all')

# import pandas_profiling
# profile = df.profile_report(title="Pandas Profiling Report", progress_bar=True)
# profile

,First Name,Last Name,E-Mail,Phone,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Organizer,Title,Organization,Membership
0,Tayjua,Hines,jhines30@kent.edu,NaN,nan,Kent State,1-Prospect,No,No,Me,"Current President of BUS, former member of the...",Akii,NaN,OSA,OSA: Columbus
1,Maggie,Ash,maggie@ignitenational.org,33070474580,nan,OSU,NaN,No,No,Me,"Recent graduate from OSU, currently a fellow w...",Akii,NaN,OSA,OSA: Columbus
2,Kelsey,Lowman,Kelsey@ignitenational.org,NaN,nan,OSU,1-Prospect,No,No,Me,"Third year at OSU, heavily involved with USG a...",Akii,NaN,OSA,OSA: Columbus
3,Raga,Maddela,maddela.3@buckeyemail.osu.edu,NaN,nan,OSU,NaN,No,No,Akii,Graduate student at OSU that was interested in...,Akii,NaN,OSA,OSA: Columbus
4,Johnny,Zhang,zhang.10080@buckeyemail.osu.edu,NaN,nan,OSU,NaN,No,No,NaN,NaN,Akii,NaN,OSA,OSA: Columbus


# Counts

In [20]:
#Overall Column counts
df.describe(include='all').loc[['count', 'unique', 'top', 'freq'], :]

,First Name,Last Name,E-Mail,Phone,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Organizer,Title,Organization,Membership
count,1357,1312,1084,549,1386,815,1118,1386,1386,602,465,1386,59,898,898
unique,915,1003,1038,533,123,330,4,2,2,119,230,15,17,4,7
top,Sarah,Smith,allanka@miamioh.edu,21635840270,nan,Safer Heights,1-Prospect,No,No,Daniel,ldrshp smmt,Edwin,Pastor,BFO,BFO: Dayton
freq,10,17,3,3,1046,42,641,1292,1294,116,73,364,31,364,364


In [21]:
#Column counts by organizer, in order of how many first names they have
df.groupby('Organizer').count().sort_values('First Name', ascending=False)

,First Name,Last Name,E-Mail,Phone,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Title,Organization,Membership
Organizer,,,,,,,,,,,,,,
Edwin,364,364,361,242,364,189,262,364,364,215,149,0,364,364
Rachael,267,267,233,49,267,252,262,267,267,142,31,0,267,267
Jacob,205,204,146,69,205,198,202,205,205,22,59,0,0,0
Robin,112,111,66,76,112,0,70,112,112,0,105,0,0,0
Daniel,92,88,115,0,118,1,117,118,118,117,0,0,118,118
Reegie,45,46,25,37,48,36,48,48,48,42,20,15,0,0
Kalesha,43,38,37,32,43,43,43,43,43,38,29,0,43,43
Tami,40,27,10,0,40,0,19,40,40,0,18,0,40,40
Kalia,38,36,36,24,38,23,34,38,38,10,16,0,0,0


In [22]:
#Cincinnati counts
print('Cincinnati counts: \n')
print(df.loc[df['Membership'].str.contains('Cincinnati', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Cincinnati', na=False), 'Membership'].value_counts().sum())

Cincinnati counts: 

AMOS: Cincinnati    118
Name: Membership, dtype: int64


Total: 
118


In [23]:
#Cleveland counts
print('Cleveland counts: \n')
print(df.loc[df['Membership'].str.contains('Cleveland', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Cleveland', na=False), 'Membership'].value_counts().sum())

Cleveland counts: 

OSA: Cleveland    267
Name: Membership, dtype: int64


Total: 
267


In [24]:
#Columbus counts
print('Columbus counts: \n')
print(df.loc[df['Membership'].str.contains('Columbus', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Columbus', na=False), 'Membership'].value_counts().sum())

Columbus counts: 

OSA: Columbus     34
AMOS: Columbus    32
Name: Membership, dtype: int64


Total: 
66


In [25]:
#Columbus|Cincinnati|Cleveland|Dayton counts
print('Columbus counts: \n')
print(df.loc[df['Membership'].str.contains('Columbus|Cincinnati|Cleveland|Dayton', na=False), 'Membership'].value_counts())

print('\n\nTotal: ')
print(df.loc[df['Membership'].str.contains('Columbus|Cincinnati|Cleveland|Dayton', na=False), 'Membership'].value_counts().sum())

Columbus counts: 

BFO: Dayton         364
OSA: Cleveland      267
AMOS: Cincinnati    118
OSA: Dayton          43
OSA: Columbus        34
AMOS: Columbus       32
Name: Membership, dtype: int64


Total: 
858


In [26]:
#Column counts by "recruited by", in order of how many names they have
df.groupby('Recruited By').count().sort_values('First Name', ascending=False)

,First Name,Last Name,E-Mail,Phone,Zip Code,Institution,Leadership Status,Core Team,Relational Organizing Leader,Notes & Reflections,Organizer,Title,Organization,Membership
Recruited By,,,,,,,,,,,,,,
Daniel,90,86,114,0,116,1,116,116,116,0,116,0,116,116
MCJC L&L,73,73,73,20,73,42,73,73,73,3,73,0,73,73
Edwin Fuller,42,42,42,37,42,26,42,42,42,11,42,0,42,42
Rachael,37,37,35,12,37,31,37,37,37,2,37,0,37,37
Rachael,34,34,33,20,34,34,34,34,34,4,34,0,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Evan O'Reilly,1,1,1,1,1,1,1,1,1,0,1,0,1,1
Evan,1,1,1,0,1,1,1,1,1,0,1,0,1,1
Erica,1,1,1,1,1,0,1,1,1,0,1,0,1,1


In [27]:
#Column counts by "Leadership Status", in order of how many names they have
df.groupby('Leadership Status').count().sort_values('First Name', ascending=False)

,First Name,Last Name,E-Mail,Phone,Zip Code,Institution,Core Team,Relational Organizing Leader,Recruited By,Notes & Reflections,Organizer,Title,Organization,Membership
Leadership Status,,,,,,,,,,,,,,
1-Prospect,612,599,483,190,641,378,641,641,291,165,641,22,349,349
2-Volunteer,277,273,258,153,277,148,277,277,186,75,277,3,236,236
3-Member Leader,151,150,118,46,151,133,151,151,90,50,151,11,123,123
4-Super Leader,49,47,36,25,49,38,49,49,26,13,49,5,28,28


In [28]:
df['Leadership Status'].value_counts()

1-Prospect         641
2-Volunteer        277
3-Member Leader    151
4-Super Leader      49
Name: Leadership Status, dtype: int64

In [29]:
df['Leadership Status'].value_counts().sum()

1118

# Export Leaders and Duplicates

In [30]:
# Output your df as an Excel document
# pd_writer = pd.ExcelWriter('OOC 2021 Leaders Cleaned.xlsx')
# df.to_excel(pd_writer, index=False, index_label=None, sheet_name="Master Sheet", encoding='UTF-8')
# pd_writer.save()
# pd_writer.close()

#Output your df as a csv
df.to_csv('OOC 2022 Leaders Cleaned_jan4_2022.csv', index=False)

In [31]:
#Export duplicates, have organizers combine them as best they can and dedup from there?
# pd_writer = pd.ExcelWriter('OOC 2021 Leaders Duplicates.xlsx')
# duplicates.to_excel(pd_writer, index=False, index_label=None, encoding='UTF-8')
# pd_writer.save()
# pd_writer.close()

# Export duplicates as csv
# duplicates.to_csv('OOC 2022 Leaders Duplicates.csv', index=False)